In [2]:
import numpy as np # library to handle data in a vectorized manner
import time
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### 2. Data Section:

#### Description of the data and its sources that will be used to solve the problem

#### Description of the Data:

The following data is required to answer the issues of the problem:

List of Boroughs and neighborhoods of Manhattan with their geodata (latitude and longitude)
List of Subway metro stations in Manhattan with their relative location
List of apartments for rent in Manhattan area with their addresses and price
Preferably, a list of apartments for rent with additional information, such as price, address, area, # of beds
Venues for each Manhattan neighborhoods ( that can be clustered)
Venues for subway metro stations

#### How the data will be used to solve the problem

The data will be used as follows:

Use Foursquare and geopy data to map top 10 venues for all Manhattan neighborhoods and clustered in groups ( as per Course LAB)
Use foursquare and geopy data to map the location of subway metro stations , separately and on top of the above clustered map in order to be able to identify the venues and amenities near each metro station, or explore each subway location separately
Use Foursquare and geopy data to map the location of rental places, in some form, linked to the subway locations.
create a map that depicts, for instance, the average rental price per square ft, around a radious of (1.5 km) around each subway station - or a similar metric. I will be able to quickly point to the popups to know the relative price per subway area.
Addresses from rental locations will be converted to geodata( lat, long) using Geopy-distance and Nominatim.
Data will be searched in open data sources if available, from real estate sites if open to reading, libraries or other government agencies such as Metro New York MTA, etc.
The procesing of these DATA will allow to answer the key questions to make a decision:

what is the cost of rent (per square ft) around a mile radius from each subway metro station?
what is the area of Manhattan with best rental pricing that meets criteria established?
What is the distance from work place ( Park Ave and 53 rd St) and the tentative future home?
What are the venues of the two best places to live? How the prices compare?
How venues distribute among Manhattan neighborhoods and around metro stations?
Are there tradeoffs between size and price and location?
Any other interesting statistical data findings of the real estate and overall data.

### Reference of venues around current residence in Melbourne for comparison to Manhattan place

In [3]:
# Riverside Quay Southbank (Riverside Quay) Southbank VIC 3006 Australia
address = 'Riverside Quay, Southbank'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Melbourne home are {}, {}.'.format(latitude, longitude))

/Users/muustafaqureshii/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  after removing the cwd from sys.path.


The geograpical coordinate of Melbourne home are -37.8210248, 144.965629.


In [4]:
neighborhood_latitude=-37.8212898
neighborhood_longitude=144.9644173

In [5]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
CLIENT_ID = '4H0VU4T30F5N1STRDOVGKOZSD2SYVZHU4LJJFAVTXUUETTQO' # My Foursquare ID
CLIENT_SECRET = 'X5CYNQD03LMAYUQNXYVHWUSI3D24LHWL1MN12OVWN5HKJP3K' # My Foursquare Secret
VERSION = '20180604'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CETGLD3C3BHQLWQYAVNIGTTHRNOCSMO3GDQ1KYN3V0RPQZVA
CLIENT_SECRET:X5CYNQD03LMAYUQNXYVHWUSI3D24LHWL1MN12OVWN5HKJP3K


In [6]:
url= 'https://api.foursquare.com/v2/venues/explore?&client_id=CETGLD3C3BHQLWQYAVNIGTTHRNOCSMO3GDQ1KYN3V0RPQZVA&client_secret=X5CYNQD03LMAYUQNXYVHWUSI3D24LHWL1MN12OVWN5HKJP3K&v=20180604&ll=-37.8212898,144.9644173&radius=500&limit=100'
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=CETGLD3C3BHQLWQYAVNIGTTHRNOCSMO3GDQ1KYN3V0RPQZVA&client_secret=X5CYNQD03LMAYUQNXYVHWUSI3D24LHWL1MN12OVWN5HKJP3K&v=20180604&ll=-37.8212898,144.9644173&radius=500&limit=100'

In [7]:
results = requests.get(url).json()
#results

In [8]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [9]:
venues = results['response']['groups'][0]['items']
    
SBnearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
SBnearby_venues =SBnearby_venues.loc[:, filtered_columns]

# filter the category for each row
SBnearby_venues['venue.categories'] = SBnearby_venues.apply(get_category_type, axis=1)

# clean columns
SBnearby_venues.columns = [col.split(".")[-1] for col in SBnearby_venues.columns]

SBnearby_venues.head(10)

/Users/muustafaqureshii/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Southbank Promenade,Pedestrian Plaza,-37.819959,144.965467
1,Ponyfish Island,Bar,-37.819918,144.965021
2,Yarra River,River,-37.819684,144.965115
3,The Langham,Hotel,-37.820370,144.965710
4,Broad Bean Organic Grocer,Grocery Store,-37.822588,144.966912
5,Arbory Bar & Eatery,Bar,-37.819020,144.966004
6,Candela Nuevo,Cocktail Bar,-37.818614,144.962604
7,Crown Promenade,Plaza,-37.821669,144.960354
8,Arbory Afloat,Bar,-37.819430,144.966338
9,Melbourne Recital Centre,Performing Arts Venue,-37.823685,144.967599


### Map of Southbank in Melbourne with venues near residence place - for reference

In [10]:
# create map of Southbank Melbourne using latitude and longitude values
map_sb = folium.Map(location=[latitude, longitude], zoom_start=20)

# add markers to map
for lat, lng, label in zip(SBnearby_venues['lat'], SBnearby_venues['lng'], SBnearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=4,
        radius=10,
        popup=label,
        color='blue',
        fill_color='#0f0f0f',
        fill_opacity=0.7,
    ).add_to(map_sb)  
    
map_sb